In [2]:
import pandas as pd
import os
# Read the DataFrames
all_data_df = pd.read_csv("../Data/raw_data_df.csv")

#Get the ISO-Codes
iso = pd.read_csv("../Data/ISO/ISO.csv")

C:\Users\nilsl\AppData\Local\Temp\ipykernel_39960\1923051193.py:4: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data_df = pd.read_csv("../Data/raw_data_df.csv")


In [3]:
#Look at all the columns
all_data_df.columns

Index(['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth', 'period',
       'reporterCode', 'reporterISO', 'reporterDesc', 'flowCode', 'flowDesc',
       'partnerCode', 'partnerISO', 'partnerDesc', 'partner2Code',
       'partner2ISO', 'partner2Desc', 'classificationCode',
       'classificationSearchCode', 'isOriginalClassification', 'cmdCode',
       'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc',
       'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty',
       'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty',
       'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt',
       'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue',
       'legacyEstimationFlag', 'isReported', 'isAggregate'],
      dtype='object')

In [4]:
#Reduce to relevant columns
data = all_data_df[["period","reporterISO", "partnerISO", "flowDesc", "cmdCode", "primaryValue"]]
data.head()

,period,reporterISO,partnerISO,flowDesc,cmdCode,primaryValue
0,1989,BRA,USA,Import,284690,147608.0
1,1989,BRA,GBR,Import,284690,4849.0
2,1989,BRA,SUN,Import,284690,220662.0
3,1989,BRA,CHE,Import,284690,1719.0
4,1989,BRA,NLD,Import,284690,6081.0


In [5]:
#Getting rid of all Reexports and Reimports
data = data[(data["flowDesc"] == "Import") | (data["flowDesc"] == "Export")]

#Getting rid of all total exports and imports to the World (W00)
data = data[(data["reporterISO"] != "W00") & (data["partnerISO"] != "W00")]

In [6]:
#Look at the HS Codes
data["cmdCode"].unique()

array([284690, 284610, 850511, 253090, 280530])

In [8]:
#Define capabilities
c_1 = [253090]
c_2 = [280530, 284610, 284690]
c_3 = [850511]

In [10]:
#Make the data for the Tradenetworks
TN_1 = data[data['cmdCode'].isin(c_1)].reset_index(drop=True)
TN_2 = data[data['cmdCode'].isin(c_2)].reset_index(drop=True)
TN_3 = data[data['cmdCode'].isin(c_3)].reset_index(drop=True)

In [11]:
#To allow for batch processing
TN_dataframes = [TN_1, TN_2, TN_3]


In [12]:
import_df = TN_1[(TN_1['flowDesc'] == 'Import') & (TN_1['period'] == 2003)]
export_df = TN_1[(TN_1['flowDesc'] == 'Export') & (TN_1['period'] == 2003)]


In [13]:
import_df[import_df['reporterISO'] == 'AUS']

,period,reporterISO,partnerISO,flowDesc,cmdCode,primaryValue
28095,2003,AUS,CYP,Import,253090,6862.0
28096,2003,AUS,URY,Import,253090,282.0
28097,2003,AUS,LAO,Import,253090,521.0
28098,2003,AUS,PNG,Import,253090,1222.0
28099,2003,AUS,FIN,Import,253090,6531.0
28100,2003,AUS,VNM,Import,253090,1496.0
28101,2003,AUS,PHL,Import,253090,904.0
28102,2003,AUS,DNK,Import,253090,1177.0
28103,2003,AUS,AUT,Import,253090,17662.0
28104,2003,AUS,CAN,Import,253090,5212.0


In [14]:
export_df[(export_df['reporterISO'] == 'CHN') & (export_df['partnerISO'] == 'AUS')]

,period,reporterISO,partnerISO,flowDesc,cmdCode,primaryValue
28808,2003,CHN,AUS,Export,253090,210011.0


In [15]:

# Dictionary to store all W_ij DataFrames for each TN_dataframe
all_W_ij_dataframes = {}

for i, current_tn_df in enumerate(TN_dataframes):
    # List to store the results for the current dataframe
    results = []

    # Loop through unique periods in the current dataframe
    for t in current_tn_df['period'].unique():
        import_df = current_tn_df[(current_tn_df['flowDesc'] == 'Import') & (current_tn_df['period'] == t)]
        export_df = current_tn_df[(current_tn_df['flowDesc'] == 'Export') & (current_tn_df['period'] == t)]

        # Get all unique reporterISO and partnerISO values that appear in either import or export for the current period
        all_reporters = pd.concat([import_df['reporterISO'], export_df['reporterISO']]).unique()
        all_partners = pd.concat([import_df['partnerISO'], export_df['partnerISO']]).unique()

        # Iterate through all possible reporter-partner combinations within the period
        for reporter_iso in all_reporters: # Renamed 'i' to 'reporter_iso' to avoid conflict with outer loop 'i'
            for partner_iso in all_partners: # Renamed 'j' to 'partner_iso' to avoid conflict with outer loop 'j'
                # Get import data from reporter_iso to partner_iso
                w_ij_export = export_df[(export_df['reporterISO'] == partner_iso) & (export_df['partnerISO'] == reporter_iso)]
 
                # Get export data from partner_iso to reporter_iso (which is effectively import for reporter_iso from partner_iso)
                w_ji_import = import_df[(import_df['reporterISO'] == reporter_iso) & (import_df['partnerISO'] == partner_iso)]

                # Calculate sum of primary values, handling cases where DataFrames might be empty
                export_value = w_ij_export["primaryValue"].sum() if not w_ij_export.empty else 0
                import_value = w_ji_import["primaryValue"].sum() if not w_ji_import.empty else 0
                
                # Only calculate W_ij if there's any trade data (import or export) for the pair
                if import_value != 0 or export_value != 0:
                    W_ij = (export_value + import_value) / 2
                    # Calculate the inconsistency as the absolute difference between import and export values
                    inconsistency = abs(import_value - export_value)

                    # Append the results to our list
                    results.append({
                        'period': t,
                        'reporterISO': reporter_iso,
                        'partnerISO': partner_iso,
                        'W_ij': W_ij,
                        'inconsistency': inconsistency
                    })

    # Create the final DataFrame for the current TN_dataframe and store it in the dictionary
    df_name = f"W_{i+1}_ij" # Using i+1 for 1-based naming convention (W_1_ij, W_2_ij, etc.)
    all_W_ij_dataframes[df_name] = pd.DataFrame(results)



# Display the resulting DataFrames
for df_name, df_content in all_W_ij_dataframes.items():
    print(f"\nDataFrame: {df_name}")
    print(df_content)


DataFrame: W_1_ij
       period reporterISO partnerISO        W_ij  inconsistency
0        1989         BRA        USA  190432.500      380865.00
1        1989         BRA        ZWE   88225.500      176451.00
2        1989         BRA        IND    3082.000        6164.00
3        1989         BRA        NOR   10175.000       20350.00
4        1989         BRA        NLD    7962.000       15924.00
...       ...         ...        ...         ...            ...
81348    2024         UZB        CZE    1990.000        3980.00
81349    2024         UZB        THA  239125.500      478251.00
81350    2024         UZB        KAZ   95713.000      191426.00
81351    2024         UZB        KGZ    8550.000        4228.00
81352    2024         UZB        TJK   72829.495      145658.99

[81353 rows x 5 columns]

DataFrame: W_2_ij
       period reporterISO partnerISO         W_ij  inconsistency
0        1989         BRA        USA  285620.0000     571240.000
1        1989         BRA        GBR  

In [18]:
print("Saving W_ij DataFrames to CSV files...")
csv_filenames = []
for df_name, df_content in all_W_ij_dataframes.items():
    filename = f"{df_name}.csv"
    try:
        df_content.to_csv(filename, index=False)
        csv_filenames.append(filename)
        print(f"Saved {filename}")
    except Exception as e:
        print(f"Error saving {filename}: {e}")


Saving W_ij DataFrames to CSV files...
Saved W_1_ij.csv
Saved W_2_ij.csv
Saved W_3_ij.csv


In [19]:
csv_filenames

['W_1_ij.csv', 'W_2_ij.csv', 'W_3_ij.csv']